<a href="https://colab.research.google.com/github/kaniarasann/Langchain/blob/main/06_Orchestration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
#!pip install langchain langgraph langchain_groq pydantic

In [47]:
from typing import Annotated, List
from pydantic import BaseModel, Field
import operator
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langgraph.constants import Send
from langgraph.graph import StateGraph, START, END

In [48]:
from langchain_groq import ChatGroq
from google.colab import userdata
groq_key = userdata.get('groq_key')

In [49]:
llm = ChatGroq(api_key=groq_key, model="llama-3.3-70b-versatile",)

In [50]:
class Section(BaseModel):
    name: str = ''
    description: str = ''

class Sections(BaseModel):
    sections: List[Section]

class State(BaseModel):
    topic: str
    sections: List[Section] = []                          # <- no reducer
    completed_sections: Annotated[List[str], operator.add] = []
    final_report: str = ""

class WorkerState(BaseModel):
    section: Section
    completed_sections: Annotated[List[str], operator.add] = []

In [51]:
planner = llm.with_structured_output(Sections)

In [52]:
def orchestrate(state: State):
    prompt = PromptTemplate.from_template(
        "Generate a detailed section plan for a report titled '{topic}'."
    ).format(topic=state.topic)
    plan: Sections = planner.invoke(prompt)
    return {"sections": plan.sections}  # <- LIST, not wrapper

In [53]:
def llm_call(state):
    prompt = PromptTemplate.from_template(
        "Write a Markdown section called '{name}'. Description: {description}"
    ) | llm
    md = prompt.invoke(
        {"name": state['section'].name, "description": state['section'].description}
    )
    return {"completed_sections": [md.content]}

In [54]:
def assign_workers(state: State):
    return [Send("llm_call", {"section": s}) for s in state.sections]

In [55]:
def synthesizer(state: State):
    body = "\n\n---\n\n".join(state.completed_sections)
    return {"final_report": f"# Report on {state.topic}\n\n{body}"}

In [56]:
graph = StateGraph(State)
graph.add_node("orchestration", orchestrate)
graph.add_node("llm_call", llm_call)
graph.add_node("synthesizer", synthesizer)

graph.add_edge(START, "orchestration")
graph.add_conditional_edges("orchestration", assign_workers, ["llm_call"])
graph.add_edge("llm_call", "synthesizer")
graph.add_edge("synthesizer", END)

builder = graph.compile()

In [57]:
result = builder.invoke({"topic": "History of Tamils"})
print(result["final_report"])

# Report on History of Tamils

### Introduction
This report provides a comprehensive overview of the current state of affairs, highlighting key trends, challenges, and opportunities. The significance of this report lies in its ability to inform stakeholders, guide decision-making, and facilitate strategic planning. By examining the latest developments and insights, this report aims to equip readers with a deeper understanding of the complex issues at play, ultimately contributing to more effective solutions and improved outcomes.

---

### Ancient Tamil Civilization
The ancient Tamil civilization, which flourished from ancient times to the 5th century CE, was a rich and vibrant culture that played a significant role in shaping the history of the Indian subcontinent. The Tamil people, who inhabited the southern part of the Indian peninsula, developed a unique and distinct civilization that was characterized by its own language, literature, art, architecture, and traditions.

#### Early 